#*WhereIsWaldo* Project - Step 2
##Author: Ernest Vereycken

### **read this before running the notebook**
- Be carefull when running cells multiple times, this may cause additional data to be added to a variable defined in a higher cell, this may cause incorrect results.
- Intermediate weights and parameters are temporarily stored in model.pth (and also read back in). If you start training again on a new data split (because of randomness) without removing the saved model.pth, the weights and parameters of the previous training will still be captured in the model.
- This is not the final version of Step 2. I will continue working on this once the model of step 1 is retrained.
- When things behave strangely, use 'Disconnect and delete runtime' and try again.
- If you get a cuda-related error, switch to cpu and run again, this will give you a more precise error message.

In [ ]:
import os
import random
from tqdm import tqdm
from PIL import Image

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import DatasetFolder, ImageFolder
from torchvision.transforms.functional import to_pil_image


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")   # use GPU if available

## First we define all classes from [**Step 1**](https://github.com/TentaTonny/WhereIsWaldo_Team18/blob/main/Pretraining_Triplet_Loss.ipynb). We need this to load pretrained model.

In [ ]:
class ViT(nn.Module):
    def __init__(self, image_size, patch_size, num_classes, dim, num_layers, heads, dropout):
        super().__init__()

        self.patchify = Patchify(image_size, patch_size, dim)
        self.pos_encoding = nn.Parameter(torch.randn(1, self.patchify.num_patches, dim))
        self.transformer = Transformer(dim, num_layers, heads, dropout = dropout)
        self.classifier = nn.Linear(dim, num_classes)

    def forward(self, x):
        x = self.patchify(x)         # (batch_size, N, dim)
        x = x + self.pos_encoding    # (batch_size, N, dim)
        x = self.transformer(x)      # (batch_size, N, dim)
        x = x.mean(1)                # (batch_size, dim)
        x = self.classifier(x)
        return x

class Patchify(nn.Module):
    def __init__(self, img_size, patch_size, embed_dim):
        super().__init__()
        self.num_patches = (img_size // patch_size) * (img_size // patch_size)
        self.img_size = img_size
        self.patch_size = patch_size
        self.embed_dim = embed_dim

        self.proj = nn.Conv2d(in_channels = 3,
                              out_channels = embed_dim,
                              kernel_size = patch_size,
                              stride = patch_size)

    def forward(self, x):
        B, C, H, W = x.shape
        x = self.proj(x)   # (batch_size, dim, img_size // patch_size, img_size // patch_size)
        # the input to the transformer should be of shape (batch_size, num_patches, embedding dim)
         # (batch_size, dim, img_size // patch_size, img_size // patch_size) --> flattten --> (batch_size, dim, num_patches) --> tranpose --> (batch_size, num_patches, dim)
        x = x.flatten(2).transpose(1,2)
        return x

class Transformer(nn.Module):
    def __init__(self, dim, num_layers, heads, dropout = 0.):

        super().__init__()

        self.norm = nn.LayerNorm(dim)
        self.layers = nn.ModuleList([])

        for _ in range(num_layers):
            self.layers.append(nn.ModuleList([
                MultiHeadAttention(dim, heads = heads, dropout = dropout),
                FeedForward(dim, dropout = dropout)]))

    def forward(self, x):
        for attn, ff in self.layers:
            x = attn(x) + x
            x = ff(x) + x

        return self.norm(x)

class MultiHeadAttention(nn.Module):

    def __init__(self, dim, heads = 8, dropout = 0.):
        super().__init__()
        self.dim = dim
        self.heads = heads

        self.query = nn.Linear(dim, dim)
        self.keys = nn.Linear(dim, dim)
        self.values = nn.Linear(dim, dim)

        self.norm = nn.LayerNorm(dim)

        self.out_proj = nn.Linear(dim, dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # N is the total number of patches
        B, N, C = x.shape

        x = self.norm(x)         # (B, N, C)

        query = self.query(x)    # (B, N, C)
        key = self.keys(x)       # (B, N, C)
        value = self.values(x)   # (B, N, C)

        dim_head = C // self.heads

        # Split (B,N,C) into (B, N, num_heads, dim_head) and permute heads which yields a shape of (B, num_heads, N, dim_head)
        # each of the heads, should have (N, dim_head)
        query = query.reshape(B, N, self.heads, dim_head).permute(0,2,1,3)
        key = key.reshape(B, N, self.heads, dim_head).permute(0,2,1,3)
        value = value.reshape(B, N, self.heads, dim_head).permute(0,2,1,3)

        # (B, num_heads, N, dim_head) with (B, num_heads, N, dim_head) --> (B, num_heads, N, N)
        attention_scores = torch.matmul(query, key.transpose(-1, -2))

        scale = dim_head ** -0.5   # (1 / sqrt(dim_head))
        attention_scores = attention_scores * scale
        attention_scores = F.softmax(attention_scores, dim = -1) # (B, num_heads, N, N)
        attention_scores = self.dropout(attention_scores)

        # extract the values
        # (B, num_heads, N, N) matmul (B, num_heads, N, dim_head) --> (B, num_heads, N, dim_head)
        out = torch.matmul(attention_scores, value)

        # (B, num_heads, N, dim_head) --> (B, N, num_heads, dim_head) --> (B, N, C)
        out = out.permute(0,2,1,3).flatten(2)   # or we can use .reshape(B, N, -1) rather than .flatten(2)
        out = self.out_proj(out)
        return out

class FeedForward(nn.Module):
    def __init__(self, dim, dropout = 0.):
        super().__init__()

        self.net = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, dim * 4),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(dim * 4, dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

## Preparing data


### Clone data directly into session.

**path:**    /content/Hey-Waldo/

**notes:**
- data will be deleted when you end session but you can just run this again.(this only takes a couple of seconds)
- you can also upload waldo data to drive but I think this is easier/cleaner

In [ ]:
!cd '/content/'
!rm -rf '/content/Hey-Waldo/'
!git clone https://github.com/vc1492a/Hey-Waldo.git

### some auxiliary methods.

In [ ]:
def show_tensor(t):
  plt.imshow(np.transpose(t, (1, 2, 0)))

def waldo_percentage(dataset):
  return round(dataset.count_waldos() / len(dataset.labels) * 100, 2)

### Create Dataset class for waldo data
#### we store image paths and their corresponding labels (0 if image does not contain waldo, 1 if image contains waldo)

In [ ]:
class WaldoBinaryDataset(Dataset):
    def __init__(self, imgs=None, labels=None,  folder_paths=None, transform=None):
        self.imgs = []
        self.labels = []
        self.transform = transform
        self.augmentation = transforms.Compose([transforms.RandomRotation(degrees=30),  #random rotation between -30 and 30 degrees
                                                transforms.RandomHorizontalFlip(p=0.5),  #0.5 chance of horizontal flipping
                                                transforms.RandomAdjustSharpness(p=0.5, sharpness_factor=4),
                                                ])
        self.is_train = True #set this to false for testset and val set

        if folder_paths:
          #(reimplement later, kan cleaner)
          for folder_path in folder_paths:
              notwaldo_path = f'{folder_path}/notwaldo'
              waldo_path = f'{folder_path}/waldo'
              #first add images with waldo
              for filename in os.listdir(waldo_path):
                file_path = os.path.join(waldo_path, filename)
                self.imgs.append(file_path)
                self.labels.append(1)

              #add images without waldo
              for filename in os.listdir(notwaldo_path):
                  file_path = os.path.join(notwaldo_path, filename)
                  self.imgs.append(file_path)
                  self.labels.append(0)

          #shuffle_data
          self.shuffle_data()


    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        image_path = self.imgs[idx]
        image = Image.open(image_path).convert("RGB")

        if self.transform is not None:
            image = self.transform(image)

        #we only perform augmentation on waldos of train set
        if self.is_train and self.labels[idx] == 1:
          image = self.augmentation(image)

        return image, self.labels[idx]

    #shuffle imgs and labels (but preserve img-label relations)
    def shuffle_data(self):
        combined = list(zip(self.imgs, self.labels))
        random.shuffle(combined)
        self.imgs, self.labels = zip(*combined)
        self.imgs = list(self.imgs)
        self.labels = list(self.labels)

    def count_waldos(self):
      return sum(self.labels)


    #static method to create subset of dataset. x included, y excluded
    @staticmethod
    def create_subset_dataset(dataset, x, y):
        subset_imgs = dataset.imgs[x:y]
        subset_labels = dataset.labels[x:y]
        subset_dataset = WaldoBinaryDataset([], transform=dataset.transform)
        subset_dataset.imgs = subset_imgs
        subset_dataset.labels = subset_labels
        subset_dataset.waldo_count = sum(subset_labels)
        subset_dataset.notwaldo_count = len(subset_labels) - subset_dataset.waldo_count

        return subset_dataset

    @staticmethod
    def duplicate_waldos(dataset, duplicate_factor):
        duplicated_images = []
        duplicated_labels = []

        for i, img in enumerate(dataset.imgs):
            if dataset.labels[i] == 1:
                for _ in range(duplicate_factor):
                    duplicated_images.append(img)
                    duplicated_labels.append(1)

        #extend original lists with the duplicated images and labels
        dataset.imgs.extend(duplicated_images)
        dataset.labels.extend(duplicated_labels)
        #shuffle set again because all the added waldos are at the end of dataset.
        dataset.shuffle_data()

### Now let's load the dataset into memory

In [ ]:
#if you use drive you should change paths here
folder_paths = ['/content/Hey-Waldo/128', '/content/Hey-Waldo/64']

#resize to 128x128
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

waldo_dataset = WaldoBinaryDataset(folder_paths=folder_paths, transform=transform)

### Split data into train-, validation- and testset

##### these ratios are defined in the assignment

In [ ]:
train_ratio = 0.81
val_ratio = 0.09
test_ratio = 0.1

tolerance = 1e-10
#we add this tolerance because of rounding errors in floating point arithmetic
if not(abs(train_ratio + val_ratio + test_ratio - 1) < tolerance):
    raise Exception("invalid train/val/test ratios")

total_length = len(waldo_dataset)
train_boundary = int(train_ratio * total_length)
validation_boundary = int((train_ratio + val_ratio) * total_length)

#split in train test and val set
train_set = WaldoBinaryDataset.create_subset_dataset(waldo_dataset, 0, train_boundary)
val_set = WaldoBinaryDataset.create_subset_dataset(waldo_dataset, train_boundary, validation_boundary)
test_set = WaldoBinaryDataset.create_subset_dataset(waldo_dataset, validation_boundary, total_length)

#disable train on test and val
test_set.is_train = True
val_set.is_train = False

#some checks
print(f'TRAIN:\nlength: {len(train_set)}. \nTotal length: {total_length}. \npercentage: {round(len(train_set) / total_length * 100.0, 2)}\nwaldos: {train_set.count_waldos()}\nwaldo percentage: {waldo_percentage(train_set)}\n')
print(f'VALIDATION:\nlength: {len(val_set)}. \nTotal length: {total_length}. \npercentage: {round(len(val_set) / total_length * 100.0, 2)}\nwaldos: {val_set.count_waldos()}\nwaldo percentage: {waldo_percentage(val_set)}\n')
print(f'TEST:\nlength: {len(test_set)}. \nTotal length: {total_length}. \npercentage: {round(len(test_set) / total_length * 100.0, 2)}\nwaldos: {test_set.count_waldos()}\nwaldo percentage: {waldo_percentage(test_set)}\n')

### Imbalance and Augmentation
As you can see, the amount of images that contain waldo is only around 1% of the total amount. When I first trained the classification model, the model just predicted 0 for every input, which resulted in a very high accuracy. Ofcourse we don't want this so we will do some data augmentation to balance out the waldo/not waldo ratio.

In [ ]:
#we duplicate every waldo n times.
duplicate_factor = 100
WaldoBinaryDataset.duplicate_waldos(train_set, duplicate_factor)

print(f'{waldo_percentage(train_set)}%')

As we can see, the ratio is now somewhere around 50%. The problem now is that we got the exact same waldo image 100 times. To fix this we add some (random) transforms, in the \_\_getitem__() method. Now when we get images to feed to the model, the images are all slightly different. The following data augmentations are applied:

```
transforms.RandomRotation(degrees=30),  
transforms.RandomHorizontalFlip(p=0.5),  
transforms.RandomAdjustSharpness(p=0.5, sharpness_factor=4),
```


Let's look at this.

**note:** if the image does not contain waldo, the augmentation will not be applied. change n untill you get an image of waldo.

In [ ]:
#get the nth image of the training set
n = 6

first_a = train_set.__getitem__(n)
first_b = train_set.__getitem__(n)
first_c = train_set.__getitem__(n)
img_a = first_a[0] #we only want the img (as tensor)
img_b = first_b[0]
img_c = first_c[0]

plt.subplot(1, 3, 1)  # 1 row, 3 columns, position 1
plt.imshow(np.transpose(img_a, (1, 2, 0)))
plt.title('img a')

# Plotting the second image
plt.subplot(1, 3, 2)  # 1 row, 3 columns, position 2
plt.imshow(np.transpose(img_b, (1, 2, 0)))
plt.title('img b')

# Plotting the third image
plt.subplot(1, 3, 3)  # 1 row, 3 columns, position 3
plt.imshow(np.transpose(img_c, (1, 2, 0)))
plt.title('img c')

plt.tight_layout()
plt.show()

In [ ]:
#creating DataLoader for each dataset
train_loader = DataLoader(train_set, batch_size=128, shuffle=True)
test_loader = DataLoader(test_set, batch_size=100, shuffle=False)
val_loader = DataLoader(val_set, batch_size=100, shuffle=False)

## Loading the pretained model
again, we could just upload the file to our drive our upload manually but I think this is easier. The github repository is public so it should work.
(if you do something similar in your own notebook, make sure you use the 'raw' URL)

In [ ]:
!cd '/content/'
!rm model_128.pth_49
!wget https://github.com/TentaTonny/WhereIsWaldo_Team18/raw/main/models/model_128.pth_49

#### We can now load the model by creating a ViT model and transfer the learned weights and parameters from the saved model

In [ ]:
pretrained_model = ViT(image_size = 128,
            patch_size = 16,
            num_classes = 2,
            dim = 512,
            num_layers = 3,
            heads = 8,
            dropout = 0.1).to(device)

checkpoint = torch.load('/content/model_128.pth_49', map_location=device)
pretrained_model.load_state_dict(checkpoint['model'])
pretrained_model.to(device)

## Check results of the pretrained model

### Plot the results of the pretrained model.

In [ ]:
train_results = []
labels = []

pretrained_model.eval()
with torch.no_grad():
    for img, label in tqdm(test_loader):
        train_results.append(pretrained_model(img.to(device)).cpu().numpy())
        labels.append(label)

train_results = np.concatenate(train_results)
labels = np.concatenate(labels)
train_results.shape

plt.figure(figsize=(15, 10), facecolor="azure")
for label in np.unique(labels):
    tmp = train_results[labels==label]
    plt.scatter(tmp[:, 0], tmp[:, 1], label=label)

plt.legend()
plt.show()

### Now try this model on some augmented waldo's


In [ ]:
#get first n waldos of train set and show them
waldo_counter = 0
i = 0
waldo_imgs = []
n = 8

while waldo_counter < n:
    img, label = train_set.__getitem__(i)
    if label == 1:
        waldo_imgs.append(img)
        waldo_counter += 1
    i += 1


plt.figure(figsize=(15,8))
plt.axis('off')

#plot the waldo's (if you change n make sure to also change the rows and columns)
for i in range(n):
  plt.subplot(2,4, i+1) #2 rows, 4 columns
  plt.imshow(np.transpose(waldo_imgs[i], (1,2,0)))
  plt.title(f'waldo_{i}')

#### We now also plot results of the pretained model on the augmented waldo's.

In [ ]:
pretrained_results = []

pretrained_model.eval()
with torch.no_grad():
    for img in waldo_imgs:
        pretrained_results.append(pretrained_model(img.to(device).unsqueeze(0)).cpu().numpy())

results = [arr[0].tolist() for arr in pretrained_results]

plt.figure(figsize=(15, 10), facecolor="azure")
for label in np.unique(labels):
    tmp = train_results[labels==label]
    plt.scatter(tmp[:, 0], tmp[:, 1], label=label)

#add results of augmented waldo's
x_values, y_values = zip(*results)

plt.scatter(x_values, y_values, color='red',label='augmented 1')
plt.legend()
plt.show()

The model does not perform well on the augmented data. One possible cause is that the model is trained on all data and not just the train set. This will be solved later.

### Anton made a new model after this. We will now have a look at this.


In [ ]:
!cd '/content/'
!rm model_final.pth_12
!wget https://github.com/TentaTonny/WhereIsWaldo_Team18/raw/main/models/model_final.pth_12

In [ ]:
pretrained_model = ViT(image_size = 128,
            patch_size = 16,
            num_classes = 2,
            dim = 512,
            num_layers = 3,
            heads = 8,
            dropout = 0.1).to(device)

checkpoint = torch.load('/content/model_final.pth_12', map_location=device)
pretrained_model.load_state_dict(checkpoint['model'])
pretrained_model.to(device)

In [ ]:
pretrained_results = []

pretrained_model.eval()
with torch.no_grad():
    for img in waldo_imgs:
        pretrained_results.append(pretrained_model(img.to(device).unsqueeze(0)).cpu().numpy())

results = [arr[0].tolist() for arr in pretrained_results]

plt.figure(figsize=(15, 10), facecolor="azure")
for label in np.unique(labels):
    tmp = train_results[labels==label]
    plt.scatter(tmp[:, 0], tmp[:, 1], label=label)

#add results of augmented waldo's
x_values, y_values = zip(*results)

plt.scatter(x_values, y_values, color='red',label='augmented 1')
plt.legend()
plt.show()

## Binary classification

In [ ]:
class WaldoBinaryClassifier(nn.Module):
    def __init__(self, pretrained):
        super().__init__()
        self.pretrained = pretrained.to(device)  #pretrained transformer (with triple loss)
        self.pretrained.classifier = nn.Linear(pretrained.classifier.in_features, 1).to(device)

    def forward(self, x):
        return torch.sigmoid(self.pretrained(x))  #sigmoid activation for binary classification

In [ ]:
binmodel = WaldoBinaryClassifier(pretrained_model)

### define training

In [ ]:
learning_rate = 0.001

bin_epochs = 20
print_every = 20

bin_criterion = nn.BCELoss()  #cross entropy loss (binary)
bin_optimizer = torch.optim.Adam(binmodel.parameters(), lr=learning_rate)
#bin_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(bin_optimizer, T_max=bin_epochs)

def train(epoch):
    binmodel.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs = inputs.to(device)
        targets = targets.float().unsqueeze(1)
        targets = targets.to(device)

        bin_optimizer.zero_grad()
        outputs = binmodel(inputs)
        loss = bin_criterion(outputs, targets)
        loss.backward()
        bin_optimizer.step()

        predictions = outputs.detach().cpu().numpy()
        predicted_labels = []
        #maybe I will write this cleaner later
        for p in predictions:
          if p[0] < 0.5:
            predicted_labels.append(0)
          else:
            predicted_labels.append(1)

        truth_labels = [1 if x.item() == 1 else 0 for row in targets for x in row]

        #count how much predictions are right
        count_equal = sum(a == b for a, b in zip(predicted_labels, truth_labels))

        train_loss += loss.item()

        total += targets.size(0)
        correct += count_equal

        epoch_loss = train_loss/(batch_idx+1)
        epoch_acc = 100.*(correct/total)

        if batch_idx % print_every == 0:
            print('Epoch {}/{}, Iter {}/{}, Train Loss: {:.3f}, Train Accuracy: {:.3f}'.format(epoch, bin_epochs, batch_idx, len(train_loader),
                                                                                   epoch_loss, epoch_acc))
    return epoch_loss, epoch_acc


### define validation

In [ ]:
def validate():
    binmodel.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(tqdm(val_loader)):
            inputs = inputs.to(device)
            targets = targets.float().unsqueeze(1)
            targets = targets.to(device)
            outputs = binmodel(inputs)
            loss = bin_criterion(outputs, targets)

            predictions = outputs.detach().cpu().numpy()
            predicted_labels = []
            #loop cleaner schrijven
            for p in predictions:
                if p[0] < 0.5:
                    predicted_labels.append(0)
                else:
                    predicted_labels.append(1)

            truth_labels = [1 if x.item() == 1 else 0 for row in targets for x in row]

            #count how much predictions are right
            count_equal = sum(a == b for a, b in zip(predicted_labels, truth_labels))

            total += targets.size(0)
            correct += count_equal
            val_loss += loss.item()

    acc = 100.*(correct/total)
    loss = val_loss/(batch_idx+1)
    print('Validation Accuracy: {:.3f}, Validation Loss: {:.3f}'.format(acc, loss))
    return acc

#### start training/validation loop

In [ ]:
best_acc = 0

losses = []
accuracies = []

for epoch in range(bin_epochs):
    epoch_loss, epoch_acc = train(epoch)
    losses.append(epoch_loss)
    accuracies.append(epoch_acc)
    #bin_scheduler.step()
    acc = validate()
    state = {'model': binmodel.state_dict(),
             'acc': acc,
             'epoch': epoch}
    if acc > best_acc:
        torch.save(state, 'model.pth')
        best_acc = acc

#### testing (after the model is trained)

In [ ]:
def test():
    binmodel.eval()

    results = []
    truth_labels = []


    with torch.no_grad():
        for img, label in tqdm(test_loader):
            results.append(binmodel(img.to(device)).cpu().numpy())
            truth_labels.append(label)

    predicted_labels = []
    #note: I've set the batch_size for testing to 1, if you change this, the code below will no longer work
    for batch in results:
        for res in batch:
            predicted_labels.append(round(res[0]))

    truth_labels = test_loader.dataset.labels
    err_idxs = []
    correct = 0
    correct_waldo = 0
    test_len = len(truth_labels)
    for i in range(test_len):
        if predicted_labels[i] == 1 and truth_labels[i] == 1:
          correct_waldo += 1
        if predicted_labels[i] == truth_labels[i]:
            correct += 1
        else:
            err_idxs.append(i)

    print(f'\naccuracy on test set: {round(correct/test_len, 3)}\n')


test()


### STEP 3 ###
Using the binary classifier for object detection

In [ ]:
from imutils.object_detection import non_max_suppression
import numpy as np
import argparse
import imutils
import time
import cv2
import os
import random
from tqdm import tqdm
from PIL import Image

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import DatasetFolder, ImageFolder
from torchvision.transforms.functional import to_pil_image




WIN_STEP = 32
ROI_SIZE = (128,128)
INPUT_SIZE = (128,128)

In [ ]:
!cd '/content/'
!rm -rf '/content/Hey-Waldo/'
!git clone https://github.com/vc1492a/Hey-Waldo.git

In [ ]:
from PIL import Image
from torchvision import transforms


transform = transforms.Compose([
    transforms.Resize((1024, 2048)),  # Adjust the size as needed
    transforms.ToTensor()
])

transformed_images = []
origs = []

# Load and transform all 19 images
for i in range(1, 20):
  # Open the image using PIL
  image_path = folder_path + f'/{i}.jpg'
  image = Image.open(image_path)
  orig = cv2.imread(image_path)
  orig = imutils.resize(orig, width=2048, height=1024)

  origs.append(orig)

  # Apply the transform to the image to concistently work with tensors from the beginning
  transformed_image = transform(image)
  transformed_images.append(transformed_image)


In [ ]:
def sliding_window_tensor(image, stepSize, windowSize):
    # image is a PyTorch tensor of shape [channels, height, width]
    for y in range(0, image.shape[1] - windowSize[1] + 1 , stepSize):
        for x in range(0, image.shape[2] - windowSize[0] + 1, stepSize):
            # Extract the window [channels, height, width]
            window = image[:, y:y + windowSize[1], x:x + windowSize[0]]
            yield (x, y, window)

In [ ]:
image_rois = []
image_loics = []
WIDTH, HEIGHT = 64, 64
STEP = 32
transform_resize_roi = transforms.Resize((128, 128))

for image in transformed_images:
  # regions of interest
  rois = []
  locs = []
  for (x,y,window) in sliding_window_tensor(image, STEP, (WIDTH, HEIGHT)):
    w, h = WIDTH, HEIGHT
    # the model is trained on 128x128 images (this resizing processes really slows down the process)
    roi = transform_resize_roi(window)
    # windows for the classifier
    rois.append(roi)
    # append 2 coordinates used to show rectangle
    locs.append(((x,y), (x + w, y + h)))
  image_rois.append(rois)
  image_loics.append(locs)

In [ ]:
total_results = []
with torch.no_grad():
  for image in image_rois:
    results = []
    for roi in image:
      prediction = binmodel(roi.unsqueeze(0).to(device)).cpu().numpy()
      results.append(prediction[0][0])
    total_results.append(results)


In [ ]:
from google.colab.patches import cv2_imshow
for i in range(0, len(origs)):
  im = origs[i]
  clone = im.copy()
  max_val = np.max(total_results[i])
  max_index = total_results[i].index(max_val)
  # index of max_value of results corresponds with index of locs = border box of waldo
  x = image_loics[i][max_index][0][0]
  y = image_loics[i][max_index][0][1]
  x2 = image_loics[i][max_index][1][0]
  y2 = image_loics[i][max_index][1][1]
  cv2.rectangle(clone, (x,y), (x2,y2), (0,255,0), 2)
  # show the visualization and current ROI
  cv2_imshow(clone)
  #cv2.imshow("ROI", roiOrig)
  cv2.waitKey(0)